In [2]:
import tensorflow as tf 

print("GPU Available:", tf.config.list_physical_devices('GPU'))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import numpy as np 
import pandas as pd 

def load_data_set() : 
    return pd.read_csv("train.csv")
    
data_set = load_data_set()
data_set_valuecount = data_set['Category'].value_counts()
data_set_valuecount.head()

data_set.head()
data_set["Category"].value_counts()/len(data_set)

Category
Education                    0.030622
Mechanical Engineer          0.028680
Electrical Engineering       0.028680
Consultant                   0.027485
Civil Engineer               0.027186
Sales                        0.027186
Management                   0.026962
Human Resources              0.026888
Digital Media                0.026738
Accountant                   0.026141
Java Developer               0.025991
Building and Construction    0.025767
Operations Manager           0.025767
Architecture                 0.025693
Testing                      0.025693
Business Analyst             0.025394
Aviation                     0.025394
Finance                      0.025319
SQL Developer                0.025245
Public Relations             0.025170
Health and Fitness           0.024796
Arts                         0.024796
Network Security Engineer    0.024647
DotNet Developer             0.024572
Apparel                      0.023900
Banking                      0.023452
Aut

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_idx,test_idx in split.split(data_set,data_set["Category"]) : 
    strat_train_data = data_set.loc[train_idx]
    strat_test_data = data_set.loc[test_idx]
    
strat_train_data["Category"].value_counts() / len(data_set)
strat_train_data.head()

In [63]:
strat_train_category = strat_train_data["Category"].copy()
strat_train_text = strat_train_data.drop(["Category"],axis=1)

In [ ]:
strat_train_text["Text"] = strat_train_text["Text"].str.lower()
strat_train_text["Text"]

In [ ]:
import re 
punctuation_pattern = r'[^\w\s$]'
strat_train_text["Text"] = strat_train_text["Text"].str.replace(punctuation_pattern,'',regex=True)

import spacy

nlp = spacy.load('en_core_web_sm')

def tokenize_data(text):
    doc = nlp(text)
    return [token.text for token in doc if not token.is_stop]

strat_train_text["Tokens"] = strat_train_text["Text"].apply(tokenize_data)

strat_train_text.head()


In [95]:
def lemmatize_data (text):
    doc = nlp(text)
    return [token.lemma_ for token in doc ]

strat_train_text["Tokens"] = strat_train_text["Text"].apply(lemmatize_data)


In [ ]:
import tensorflow_hub as hub 

bert_preprocessor = hub.keraslayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_model = hub.keraslayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1")

def bert_vectorization(text):
    pre_precessed_text = bert_preprocessor(text)
    return bert_model(pre_precessed_text)["pooled_output"]

strat_train_text["Tokens"] = strat_train_text["Tokens"].apply(bert_vectorization)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight 

class_weight = compute_class_weight(class_weight='balanced',classes = np.unique(data_set["Category"]),y=data_set["Category"])
class_weight_dict = dict(enumerate(class_weight))
print("Class weights", class_weight_dict)